### Example of Symbolic Regression to estimate LAI from spectral and structural data in vineyards

Data generated by Sergio Alvarez, GRAPEX project.
File 'RGBNIR_DSM_CHM_VIf_v4.csv' contains as potential predictors
- Reflectance values from canopy
- Vegetation indices for canopy
- Geometric information (H, widht, etc) per vine
- two predictants (afc, LAI) in the last colunms.

### Instructions
- if not, create a anaconda environment, e.g. pysr
- once completed, in the terminal, 
- in the pysr environment installa modules:
- conda install pysr
- conda install matplotlib

- activate environment and pass two commands:
- conda activate pysr
- python -c "import pysr
- last commands will install julia libraries, necessary to run the notebook.

Julia and Julia dependencies are installed at first import:

In [ ]:
import pysr

Now, let's import everything else as well as the PySRRegressor:


In [ ]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# Simple PySR example:


use Sergio's v4 dataset, predict LAI

In [ ]:
df = pd.read_csv('RGBNIR_DSM_CHM_VIf_v4.csv')
# print(df.head())
X = df.iloc[:,1:-3]
y = df.LAI
print(X.head())


By default, we will set up 50 populations of expressions (which evolve independently except for migrations),# use 8 threads, and use `"best"` for our model selection strategy:

In [ ]:
default_pysr_params = dict(
    populations=8,    # ^ Assuming we have 4 cores, this means 2 populations per core, so one is always running.
    population_size=50, # ^ Number of equations per population
    model_selection="best", #"score" is also an option,
    random_state=0, # this line ensure reproducible results
    maxsize=20, # maximum complexity of the equations, increase to 200 for more complex equations
    deterministic=True, #   to ensure reproducible results
    parallelism='serial', # to avoid parallel computing issues on some systems, change to 'serial' if needed
    # select_k_features=10, # to automatically select few predictors from the entire set of predictor options, activate/deactive as you wish
    # denoise=True, # to remove scatteting on y, activate/deactivate as you wish
    verbosity =0,
    elementwise_loss= "L2DistLoss()",  #  (mean square)
    #niterations=10000000,  # Run forever
    #timeout_in_seconds=3600*2,  # but stop after 2 hours
    )

PySR can run for arbitrarily long, and continue to find more and more accurate expressions. You can set the total number of cycles of evolution with `niterations`, although there are also a [few more ways](https://github.com/MilesCranmer/PySR/pull/134) to stop execution.

**This first execution will take a bit longer to startup, as the library is JIT-compiled. The next execution will be much faster.**

In [ ]:
# Learn equations
model = PySRRegressor(
    niterations=100, # number of optimization iterations
    binary_operators=["+", "*","-","/"],
    unary_operators=["sqrt", "exp", "log"],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    **default_pysr_params,
)

model.fit(X, y)

We can print the model, which will print out all the discovered expressions:

In [ ]:
print(model)

We can also view the SymPy format of the best expression:

In [ ]:
model.sympy()

We can also view the SymPy of any other expression in the list, using the index of it in `model.equations_`.

In [ ]:
model.sympy(8)

## Output

`model.equations_` is a Pandas DataFrame. We can export the results in various ways:

In [ ]:
model.latex()

These is also `model.sympy(), model.jax(), model.pytorch()`. All of these can take an index as input, to get the result for an arbitrary equation in the list.

We can also use `model.predict` for arbitrary equations, with the default equation being the one chosen by `model_selection`:

ypredict = model.predict(X)
ypredict_simpler = model.predict(X, 2)

print("Default selection MSE:", np.power(ypredict - y, 2).mean())
print("Manual selection MSE for index 2:", np.power(ypredict_simpler - y, 2).mean())

Plotting the first solutions

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(12, 12))

it = np.linspace(0,11,12, dtype=int)  # Ensure indices are integers

# Iterate over the subplots and data
for ax, i in zip(axes.flatten(), it):  # Flatten axes for proper iteration
    ypredict_simpler = model.predict(X, index=i)  # Use integer index
    ax.plot(ypredict_simpler, y,'.')
    ax.grid(True)
    # ax.set_xlabel("Modeled LAI")
    # ax.set_ylabel("Measured LAI")
    ax.plot( [0,5], [0,5], linestyle='--', color='k' )

    ax.set_aspect('equal')
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    mse =np.power(ypredict_simpler - y, 2).mean()
    ax.set_title('Equation = ' + str(i))
    ax.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()



Plotting the difference between the predicted and measured LAI

# Other PySR Options

The full list of PySR parameters can be found here: https://ai.damtp.cam.ac.uk/pysr/api